## Installation of packages

In [1]:
%pip install numpy -q
%pip install keras -q

%pip install tensorflow -q
%pip install tensorboard_plugin_profile -q

%pip install networkx -q

%pip install wandb -q
%pip install kaggle -q

%pip install matplotlib -q
%pip install ipympl -q


/bin/bash: /home/vulgrim/miniconda3/envs/tensorflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/vulgrim/miniconda3/envs/tensorflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/vulgrim/miniconda3/envs/tensorflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/vulgrim/miniconda3/envs/tensorflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/vulgrim/miniconda3/envs/tensorflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/vulgrim/minic

## Setup Tensorboard

In [10]:
from os.path    \
    import      \
    isdir,      \
    join

from os         \
    import      \
    walk,       \
    remove

from random                     \
    import                      \
    SystemRandom

import wandb

from wandb.keras                \
    import                      \
    WandbMetricsLogger,         \
    WandbCallback,              \
    WandbEvalCallback

from keras.utils                \
    import                      \
    image_dataset_from_directory

from keras.backend              \
    import                      \
    clear_session

from keras.models               \
    import                      \
    Sequential,                 \
    load_model

from keras.layers               \
    import                      \
    RandomFlip,                 \
    RandomZoom,                 \
    RandomContrast,             \
    RandomBrightness,           \
    RandomRotation

from keras.optimizers           \
    import SGD

from keras.callbacks            \
    import TensorBoard

from keras                      \
    import mixed_precision

import tensorflow

from tensorflow.python.ops      \
  import summary_ops_v2

from keras.losses \
    import SparseCategoricalCrossentropy


In [3]:
is_using_tensorboard: bool = True
location_to_tensorboard: str = '/tmp/tensorboard'

if is_using_tensorboard:
    if isdir(
        location_to_tensorboard
    ):
        for root,           \
            directories,    \
            files           \
                in walk(
                    location_to_tensorboard
                ):
            
            for file in files:
                full_path_to_file: str = join(
                    root, 
                    file
                )

                remove(
                    full_path_to_file
                )

%load_ext tensorboard
%tensorboard --logdir {location_to_tensorboard}


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 326689), started 0:00:19 ago. (Use '!kill 326689' to kill it.)

## Setup Matplotlib

In [4]:
use_ipympl: bool = False

if use_ipympl:
    %matplotlib ipympl

## Packages used

In [5]:
global_policy = mixed_precision.Policy(
    'bfloat16'
)

mixed_precision.set_global_policy(
    global_policy
)

tensorflow.keras                                        \
    .mixed_precision                                    \
    .set_global_policy(
    global_policy
)

gpu_memory_growth: bool = True

def zero() -> int:
    return 0


physical_devices = tensorflow.config.list_physical_devices(
    str(
        'gpu'
    ).upper()
)

selected_physical_device = physical_devices[
    zero()
]

tensorflow.config.experimental.set_memory_growth(
    selected_physical_device,
    gpu_memory_growth
)

In [6]:
location_of_model: str = '/opt/models/O2RM'

def get_location_of_model() -> str:
    global location_of_model
    return location_of_model


In [7]:
def load_old_model_weights():
    global          \
        model,      \
        location_of_model

    if isdir(
        location_of_model
    ):
        model.load_weights(
            location_of_model
        )

In [11]:
clear_session()

model = load_model(
    location_of_model
)

model.compile(
    optimizer=SGD(
        learning_rate=0.024
    ),
    loss=SparseCategoricalCrossentropy(
        from_logits=True
    ),
    metrics=[
        'accuracy'
    ]
)

In [12]:
def generate_seed():
    return SystemRandom().randint(
        1,
        32767
    )

In [25]:
dataset_seed: int = generate_seed()

location_of_dataset: str = '/opt/dataset/numbers'

width: int = 512
height: int = 512
channels: int = 3

number_of_labels: int = 10

batches: int = 50
epochs: int = 12

validation_split: float = 0.5

use_multiprocessing: bool = True
process_workers: int = 4

In [15]:
wandb_entity: str = 'designermadsen'
wandb_project: str = 'O2RM'

wandb_jobtype: str = 'Training'

wandb_tags: list = [
    'Nvidia',
    'Linux',
    'Ubuntu',
    'Development',
    'Random',
    'test-driven',
    'Bare-Metal',
    'TensorFlow'   
]

wandb_use_callback: bool = False
wandb_measure_metrics: bool = True
wandb_compute_flops: bool = True

In [ ]:
training_history: list = list()

In [16]:
configuration: dict = {
    'vision': 
    {
        'width': width,
        'height': height,
        'channels': channels
    },

    'dataset': 
    {
        'number of labels': number_of_labels,
        'batches': batches,
        'epochs': epochs,
        'seed': dataset_seed,
        'using multiprocessing': use_multiprocessing,
        'using tensorboard': is_using_tensorboard,
        'workers': process_workers,
    },

    'wandb':
    {
        'job type': wandb_jobtype,
        'tags': wandb_tags,
        'measure metrics': wandb_measure_metrics,
        'use callbacks': wandb_use_callback,
        'compute flops': wandb_compute_flops
    },

    'gpu':
    {
        'memory growth': gpu_memory_growth
    }
}

In [17]:
training_dataset = None
validation_dataset = None

def get_training_dataset():
    global training_dataset
    return training_dataset

def get_validation_dataset():
    global validation_dataset
    return validation_dataset

In [18]:
def setup_wandb_callback():
    global wandb_compute_flops

    return WandbCallback(
        monitor='val_loss', 
        verbose=0,
    
        save_weights_only=True,
        log_weights=True,
        log_gradients=True,
        save_graph=True,
    
        save_model=True,
    
        training_data=get_training_dataset(),
        validation_data=get_validation_dataset(),
        log_evaluation=True,
    
        compute_flops=wandb_compute_flops,    
        
        input_type='image',
        output_type='label',
        
        labels=get_training_dataset().class_names,
        predictions=15
    )            
    

In [19]:
def setup_wandb_metrics():
    return WandbMetricsLogger()

In [21]:
def setup_tensorboard():
    global location_to_tensorboard
    return TensorBoard(
        location_to_tensorboard,
        histogram_freq=0,
        write_graph=True,
        write_images=True, 
        write_steps_per_second=True,
        update_freq=True, 
        profile_batch=False,
        embeddings_freq=0
    )

In [22]:
def callbacks() -> list:
    callback_list: list = list()

    global                      \
        wandb_measure_metrics,  \
        wandb_use_callback,     \
        wandb_compute_flops,    \
        is_using_tensorboard

    if is_using_tensorboard:
        callback_list.append(
            setup_tensorboard()
        )

    if wandb_measure_metrics:
        callback_list.append(
            setup_wandb_metrics()
        )
    
    if wandb_use_callback:
        callback_list.append(
            setup_wandb_callback()
        )

    return callback_list

In [23]:
def append_training_session(
    history
) -> None:
    global training_history

    training_history.append(
        history.history
    )

In [24]:
device_name = str('/') + str(
    selected_physical_device.name[
        len(
            '/physical_device:'
        )
        :
        len(
            selected_physical_device.name
        )
    ]
)

autotune = tensorflow.data.AUTOTUNE
training_rotations: int = 1

In [29]:
def refresh_seed():
    global dataset_seed
    dataset_seed = generate_seed()

In [30]:
def load_dataset():
    global training_dataset, validation_dataset, location_of_dataset, validation_split, dataset_seed, height, width, batches

    if not(training_dataset is None)        \
       or                                   \
       not(validation_dataset is None):
        refresh_seed()

    training_dataset, validation_dataset    \
        = image_dataset_from_directory(
        location_of_dataset,
        validation_split=validation_split,
        subset='both',
        seed=dataset_seed,
        image_size=(
            height,
            width
        ),
        batch_size=batches
    )


In [31]:
wandb.init(
    entity=wandb_entity,
    project=wandb_project,
    config=configuration,
    tags=wandb_tags,
    job_type=wandb_jobtype,
    reinit=True,
    tensorboard=is_using_tensorboard,
    save_code=False,
    notes=""
)

with tensorflow.device(
    device_name
):
    load_old_model_weights()
    load_dataset()

    wandb.log(
        {
            'seed': dataset_seed,
            'training': {
                'labels': get_training_dataset().class_names
            },
            'validation': {
                'labels': get_validation_dataset().class_names
            }
        }
    )

    history = model.fit(
        training_dataset.prefetch(
            buffer_size=autotune
        ),
        
        validation_data=validation_dataset.prefetch(
            buffer_size=autotune
        ),

        epochs=epochs,

        callbacks=callbacks(),

        use_multiprocessing=use_multiprocessing,
        workers=process_workers
    )

    append_training_session(
        history
    )

    model.save(
        location_of_model,
        save_format='tf',
        overwrite=True
    )

    saved_model = wandb.Artifact(
        "o2rm_model",
        type="model"
    )

    saved_model.add_dir(
        location_of_model
    )

    wandb.log_artifact(
        saved_model
    )

model.summary()

wandb: WARNING Path /opt/programming/ORM-Model/notebooks/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path /opt/programming/ORM-Model/notebooks/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path /opt/programming/ORM-Model/notebooks/wandb/ wasn't writable, using system temp directory
wandb: Currently logged in as: designermadsen. Use `wandb login --relogin` to force relogin


2023-08-13 22:22:03.853719: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open /opt/models/O2RM: FAILED_PRECONDITION: /opt/models/O2RM; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


Found 20000 files belonging to 10 classes.


wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in /tmp/tensorboard/train


Using 10000 files for training.
Using 10000 files for validation.
Epoch 1/12


...

200/200 [==============================] - ETA: 0s - loss: 2.1719 - accuracy: 0.0889

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`
wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in /tmp/tensorboard/validation


200/200 [==============================] - 321s 1s/step - loss: 2.1719 - accuracy: 0.0889 - val_loss: 2.1719 - val_accuracy: 0.0845
Epoch 2/12
200/200 [==============================] - 291s 1s/step - loss: 2.1719 - accuracy: 0.0869 - val_loss: 2.1719 - val_accuracy: 0.0845
Epoch 3/12
200/200 [==============================] - 294s 1s/step - loss: 2.1719 - accuracy: 0.0874 - val_loss: 2.1719 - val_accuracy: 0.0845
Epoch 4/12
200/200 [==============================] - 293s 1s/step - loss: 2.1719 - accuracy: 0.0864 - val_loss: 2.1719 - val_accuracy: 0.0854
Epoch 5/12
200/200 [==============================] - 295s 1s/step - loss: 2.1719 - accuracy: 0.0859 - val_loss: 2.1719 - val_accuracy: 0.0850
Epoch 6/12
200/200 [==============================] - 298s 1s/step - loss: 2.1719 - accuracy: 0.0869 - val_loss: 2.1719 - val_accuracy: 0.0850
Epoch 7/12
200/200 [==============================] - 295s 1s/step - loss: 2.1719 - accuracy: 0.0894 - val_loss: 2.1719 - val_accuracy: 0.0850
Epoch 8/12

INFO:tensorflow:Assets written to: /opt/models/O2RM/assets
wandb: Adding directory to artifact (/opt/models/O2RM)... Done. 0.2s


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 512, 512, 3)       0         
                                                                 
 new_layer_1 (Conv2D)        (None, 512, 512, 3)       84        
                                                                 
 new_layer_2 (Conv2D)        (None, 512, 512, 3)       84        
                                                                 
 new_layer_3 (Conv2D)        (None, 512, 512, 3)       84        
                                                                 
 new_layer_4 (Conv2D)        (None, 512, 512, 3)       84        
                                                                 
 conv2d (Conv2D)             (None, 512, 512, 2)       56        
                                                                 
 conv2d_1 (Conv2D)           (None, 512, 512, 2)       3

In [32]:
load_dataset()

Found 20000 files belonging to 10 classes.
Using 10000 files for training.
Using 10000 files for validation.


In [36]:
model.evaluate(
    training_dataset.prefetch(
        buffer_size=autotune
    ),
    callbacks=[WandbMetricsLogger()]
)

model.evaluate(
    validation_dataset.prefetch(
        buffer_size=autotune
    ),
    callbacks=[WandbMetricsLogger()]
)

200/200 [==============================] - 79s 393ms/step - loss: 2.1719 - accuracy: 0.0874


[2.171875, 0.08740234375]

In [37]:
summary_ops_v2.flush()
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch/accuracy,▅▂▃▂▁▂▆▆▅▅▆█
epoch/epoch,▁▂▂▃▄▄▅▅▆▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▁▁█▅▅▅▅▅▅▅▅
epoch/val_loss,▁▁▁▁▁▁▁▁▁▁▁▁
global_step,▁▁▁▁▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████████
seed,▁
train/batch_accuracy,▄▅▃▂▅▃▁▄▄▂▂▄▂▁▅▃▁▅▃▂▄▆▃▄▆▃▂▆▃▂█▅▂▄▆▃▃▆▃▂
train/batch_loss,▄▇▄▁▇▄▂▆▅▂▄█▃▁▇▄▁▇▄▂▄█▃▄█▃▁▇▄▁▄▇▃▄█▃▄█▃▁
train/batch_steps_per_second,█████▇▇▆▇█▇▄▅▇▆▅▆▆▆▅▇▆▆█▆▇▇▁▆▆▇▆▆▇▄▆▇▇▇▅
